# Introduction to STAC Part 2:

In this tutorial, we will highlight how to create a simple STAC catalog using the python library PySTAC. We will refer back to the catalog of items we obtained from our Planet order and the corresponding analytic assets (our COGs stored in our Google Cloud Storage bucket).

In [1]:
import pystac
import os
import json
import requests
import urllib.request
import rasterio
from tempfile import TemporaryDirectory
from pathlib import Path
import typing

In [2]:
pystac.get_stac_version()

'1.0.0'

In [3]:
catalog = pystac.Catalog(id='sample-catalog', description='Simple STAC catalog.')

In [4]:
print(list(catalog.get_all_items()))
print(list(catalog.get_children()))

[]
[]


In [13]:
import requests
from requests.auth import HTTPBasicAuth
import json

## Get Planet API key stored as an environment variable on your system
PLANET_API_KEY = os.getenv('PLANET_API_KEY')

## HTTP Basic Authentication
planet_auth = HTTPBasicAuth(PLANET_API_KEY, '')

# set content type to json
headers = {'content-type': 'application/json'}

In [14]:
# Base Order URL
planet_orders_url = 'https://api.planet.com/compute/ops/orders/v2'

# order id
order_id = 'cb363b75-b4e4-46f1-8d32-0ef87afb6215'

In [16]:
item_ids = [
         '20210515_145754_03_245c',
         '20210514_145807_70_2455',
]

## Microsoft Dask

In [6]:
import numpy as np
import xarray as xr

import rasterio.features
import stackstac
import pystac_client
import planetary_computer

import xrspatial.multispectral as ms

from dask_gateway import GatewayCluster

/home/alex/github_repos/ml-notebooks/.venv/lib/python3.8/site-packages/dask_gateway/client.py:21: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import LoopRunner, format_bytes
